In [155]:
#Import necessary libraries.

from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
import numpy as np


pd.set_option('display.max_rows',500)
pd.set_option('display.max_columns',500)
pd.set_option('display.width',1000)

###### To get the urls first and the second page for each year  from 2009 to 2021

In [116]:
#since the first page and last page has the same pattern url for all the years 
urls =[]
years = [str(i) for i in range(2009,2022)] 
for year in years:
    urls.append (f"https://www.amazon.com/gp/bestsellers/{year}/books")
    urls.append(f"https://www.amazon.com/gp/bestsellers/{year}/books/ref=zg_bsar_pg_2/ref=zg_bsar_pg_2?ie=UTF8&pg=2")
    
#urls

###### This cell contains a fuction that gets the observation for each book in a year

In [3]:
def get_dir(book,year): 
    '''to get the details of each book for each''' 
    
    import numpy as np
    '''to get the name of price'''

    try:
        price = book.find('span',class_="_cDEzb_p13n-sc-price_3mJ9Z").text[1:]
    except Exception as e:
        price = np.nan
    try:
        ranks = book.find('span', class_='zg-bdg-text').text[1:]
    except Exception as e:
        ranks = np.nan
    try:
        title = book.find('div',class_="_cDEzb_p13n-sc-css-line-clamp-1_1Fn1y").text
    except Exception as e:
        title = np.nan
    try:
        ratings= book.find('span',class_="a-icon-alt").text[:3] 
    except Exception as e:
        ratings = np.nan
    try:
        no_of_reviews  = book.find('span',class_="a-size-small").text
    except Exception as e:
        no_of_reviews = np.nan
    try:
        author = book.find('a',class_="a-size-small a-link-child").text
    except Exception as e:
        author = np.nan
    try:
        cover_type = book.find('span',class_="a-size-small a-color-secondary a-text-normal").text
    except Exception as e:
        cover_type = np.nan
    year = year
    return [price,ranks,title,no_of_reviews,ratings,author,cover_type, year]

###### To get the year for the first and second page

In [ ]:
year = [(str(i),str(i)) for i in range(2009,2022)] 
years = [j for i in year for j in i] 
#years

###### To get the books in every page(first and second) of every year from year 2009- 2020
######  This cell ran for 20  minutes on my machine, so you will have to exercise patience

In [ ]:



all_year = [] 
from time import sleep 
for url in urls:  
    
    website = url 
    
    driver = webdriver.Chrome("C:/Users/USER/Desktop/Amazon scrape/chromedriver.exe")  
    
    driver.get(website)        
    
    sleep(30)                  
    
    the_soup = BeautifulSoup(driver.page_source, 'html.parser')           
    
    books = the_soup.find_all(id = 'gridItemRoot')                 
    
    all_year.append(books) 
    
    
    
    driver.quit()                      
#


In [121]:
len(all_year), len(years)  


(0, 26)

###### this cell is used to get index and year so that looping through the files will be easier

In [7]:

year_index = (list(enumerate(years)))
dc = year_index

###### This cell is for getting the observation for all the books in the top 100 for every year with the period of 2009-2022


In [ ]:
data = [] 
for i in dc:   
    for books in all_year[i[0]]:             
        for book in books:                   
            data.append(get_dir(book,i[1]))  
            


In [22]:

 
# open file
with open('Amazon.txt', 'w+') as f:
     
    # write elements of list
    for items in data: 
        try:
            f.write('%s\n' %items)
        except Exception as e:
            f.write('%s\n' 'nothing')
     
    print("File written successfully")
 
 
# close the file
f.close()

File written successfully


###### This cell is for converting to data extracted in list format to a dataframe

In [13]:
top_books= pd.DataFrame(data, columns = [
                         'price',
                         'ranks',
                         'title',
                         'no_of_reviews',
                         'ratings',
                         'author',
                       'cover_type',
                          'year'])



###### To save to csv

In [16]:
top_books.to_csv('best_selling_books_2009-2021_full_edited.csv')   #To save to csv

In [ ]:
top_books

###### The next  4 cells are to get  the  categories for our from the kaggle datset

###### Alternate method  to get  the  categories for our from the kaggle datset

In [152]:
kaggle =pd.read_csv('bestsellers with categories.csv') 
def dfghj(col):
    e = list(kaggle.Name)                

    y = list(top_books.title)            # make a list of the book title from the our dataset

    d = list(kaggle.Genre)                    # make a list of caegories from the kaggle dataset

    we = {e[i]:d[i] for i in range(550)}           # make a dictionary contain each book and its category

                                      # make an empty list to get the categories for the books in our dataset
    if col in list(e):                        # check ifour book title is in the kaggle book title

        cor = we[col]                   # return category if the check in the above line is True
    else:
        cor = ('unknown')                # return "unknown"
    
    return cor




test = top_books.copy()
test['category'] = [dfghj(i) for i in list(test.title)]
test

,price,ranks,title,no_of_reviews,ratings,author,cover_type,year,category
0,12.49,1,The Lost Symbol,"16,118",4.4,Dan Brown,Hardcover,2009,Fiction
1,13.40,2,The Shack: Where Tragedy Confronts Eternity,"23,392",4.7,William P. Young,Paperback,2009,Fiction
2,9.93,3,Liberty and Tyranny: A Conservative Manifesto,"5,036",4.8,Mark R. Levin,Hardcover,2009,Non Fiction
3,14.30,4,"Breaking Dawn (The Twilight Saga, Book 4)","16,912",4.7,Stephenie Meyer,Hardcover,2009,Fiction
4,9.99,5,Going Rogue: An American Life,"1,572",4.6,Sarah Palin,Hardcover,2009,Non Fiction
...,...,...,...,...,...,...,...,...,...
1286,16.69,96,Will,Will Smith,4.8,NaN,Hardcover,2021,unknown
1287,7.49,97,Think and Grow Rich: The Landmark Bestseller N...,"79,872",4.7,Napoleon Hill,Paperback,2021,unknown
1288,8.95,98,Dragons Love Tacos,"15,753",4.8,Adam Rubin,Hardcover,2021,unknown
1289,7.59,99,The Truth About COVID-19: Exposing The Great R...,Doctor Joseph Mercola,4.8,NaN,Hardcover,2021,unknown


In [172]:
test.head(50)

,price,ranks,title,no_of_reviews,ratings,author,cover_type,year,category
0,12.49,1,The Lost Symbol,"16,118",4.4,Dan Brown,Hardcover,2009,Fiction
1,13.40,2,The Shack: Where Tragedy Confronts Eternity,"23,392",4.7,William P. Young,Paperback,2009,Fiction
2,9.93,3,Liberty and Tyranny: A Conservative Manifesto,"5,036",4.8,Mark R. Levin,Hardcover,2009,Non Fiction
3,14.30,4,"Breaking Dawn (The Twilight Saga, Book 4)","16,912",4.7,Stephenie Meyer,Hardcover,2009,Fiction
4,9.99,5,Going Rogue: An American Life,"1,572",4.6,Sarah Palin,Hardcover,2009,Non Fiction
5,18.29,6,StrengthsFinder 2.0,Gallup,4.1,NaN,Hardcover,2009,Non Fiction
6,12.66,7,The Help,"18,068",4.8,Kathryn Stockett,Hardcover,2009,Fiction
7,17.56,8,New Moon (The Twilight Saga),"12,329",4.7,Stephenie Meyer,Paperback,2009,Fiction
8,58.90,9,The Twilight Saga Collection,"6,100",4.7,Stephenie Meyer,Hardcover,2009,Fiction
9,16.04,10,Outliers: The Story of Success,"22,209",4.7,Malcolm Gladwell,Hardcover,2009,Non Fiction


In [168]:
dff = test[['title','category']]

In [171]:
dff.to_csv('find_cat.csv')

In [115]:
#best_selling_books_2009_2019.to_csv('best_selling_books_2009-2019.csv')

In [173]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1291 entries, 0 to 1290
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   price          1252 non-null   object
 1   ranks          1291 non-null   object
 2   title          1283 non-null   object
 3   no_of_reviews  1287 non-null   object
 4   ratings        1287 non-null   object
 5   author         1109 non-null   object
 6   cover_type     1285 non-null   object
 7   year           1291 non-null   object
 8   category       1291 non-null   object
dtypes: object(9)
memory usage: 90.9+ KB


In [174]:
test.no_of_reviews

0                      16,118
1                      23,392
2                       5,036
3                      16,912
4                       1,572
                ...          
1286               Will Smith
1287                   79,872
1288                   15,753
1289    Doctor Joseph Mercola
1290                   27,332
Name: no_of_reviews, Length: 1291, dtype: object